In [ ]:
import pandas as pd
import sqlite3
import openai
import ipywidgets as widgets
from google.colab import files
from IPython.display import display

In [ ]:
from google.colab import userdata
key=userdata.get('key').strip()

In [ ]:
# OpenAI API key (Replace with your actual key)
openai.api_key = key

In [ ]:
# Standardized database schema for People Database
STANDARD_COLUMNS = {"name": "Name", "company": "Company", "phone": "Phone", "email": "Email"}

# Function to get column name suggestions from OpenAI
def suggest_column_mapping(columns):
    prompt = f"Match the following dataset columns to standardized names {STANDARD_COLUMNS}.\nColumns: {columns}\n\nReturn as JSON."
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    try:
        mapping = eval(response.choices[0].message.content)
        return mapping
    except:
        return {col: "" for col in columns}  # Default to empty mapping if error occurs

In [ ]:
# Function to insert data into SQLite database
def insert_into_database(df, mapping):
    conn = sqlite3.connect("people_database.db")
    cursor = conn.cursor()

    # Create table if not exists
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS people (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            company TEXT,
            phone TEXT,
            email TEXT
        )
    ''')

    # Rename columns as per mapping
    df.rename(columns=mapping, inplace=True)

    # Keep only required columns
    df = df[list(STANDARD_COLUMNS.keys())]

    # Insert data into SQLite
    df.to_sql("people", conn, if_exists="append", index=False)
    conn.commit()
    conn.close()
    print("Data inserted successfully.")


In [ ]:
def confirm_mapping(columns, suggestions):
    confirmed_mapping = {}
    dropdowns = {}

    print("Suggested Mappings:")

    for col in columns:
        # Get the suggested value, defaulting to the first standard column key
        suggested_value = suggestions.get(col, "")

        # Automatically match columns based on name similarity
        auto_mapped_value = next((key for key in STANDARD_COLUMNS.keys() if key.lower() in col.lower()), None)

        # Use automatic mapping if available, otherwise use OpenAI's suggestion or default
        value = auto_mapped_value if auto_mapped_value else (suggested_value if suggested_value in STANDARD_COLUMNS.keys() else list(STANDARD_COLUMNS.keys())[0])

        dropdowns[col] = widgets.Dropdown(
            options=list(STANDARD_COLUMNS.keys()),
            value=value,  # Use the validated value
            description=col,
            style={'description_width': 'initial'}
        )
        display(dropdowns[col])

    confirm_button = widgets.Button(description="Confirm Mappings")

    def on_confirm_clicked(b):
        for col in columns:
            confirmed_mapping[col] = dropdowns[col].value
        print("Mappings confirmed.")
        insert_into_database(df, confirmed_mapping)

    confirm_button.on_click(on_confirm_clicked)
    display(confirm_button)


In [ ]:
# Main function to process an uploaded file
def process_excel_file():
    print("Upload an Excel or CSV file")
    uploaded = files.upload()
    file_path = list(uploaded.keys())[0]

    global df  # Make df accessible to confirm_mapping function
    df = pd.read_excel(file_path) if file_path.endswith(('.xls', '.xlsx')) else pd.read_csv(file_path)
    columns = df.columns.tolist()

    suggestions = suggest_column_mapping(columns)
    confirm_mapping(columns, suggestions)

In [ ]:
if __name__ == "__main__":
    process_excel_file()

Upload an Excel or CSV file


Saving sample_people_data.csv to sample_people_data (2).csv
Suggested Mappings:


Dropdown(description='Full Name', options=('name', 'company', 'phone', 'email'), style=DescriptionStyle(descri…

Dropdown(description='Company Name', options=('name', 'company', 'phone', 'email'), style=DescriptionStyle(des…

Dropdown(description='Phone Number', index=2, options=('name', 'company', 'phone', 'email'), style=Description…

Dropdown(description='Email Address', index=3, options=('name', 'company', 'phone', 'email'), style=Descriptio…

Button(description='Confirm Mappings', style=ButtonStyle())

Mappings confirmed.
Data inserted successfully.


In [ ]:
# Function to view data in SQLite database
def view_database():
    conn = sqlite3.connect("people_database.db")
    df = pd.read_sql_query("SELECT * FROM people", conn)
    conn.close()
    display(df)

view_database()

,id,name,company,phone,email
0,1,Alice Johnson,TechCorp,123-456-7890,alice@techcorp.com
1,2,Bob Smith,InnoSoft,234-567-8901,bob@innosoft.com
2,3,Charlie Brown,DataWorks,345-678-9012,charlie@dataworks.com
3,4,David White,WebGen,456-789-0123,david@webgen.com
4,5,Emma Black,CloudNet,567-890-1234,emma@cloudnet.com
5,6,Alice Johnson,TechCorp,123-456-7890,alice@techcorp.com
6,7,Bob Smith,InnoSoft,234-567-8901,bob@innosoft.com
7,8,Charlie Brown,DataWorks,345-678-9012,charlie@dataworks.com
8,9,David White,WebGen,456-789-0123,david@webgen.com
9,10,Emma Black,CloudNet,567-890-1234,emma@cloudnet.com
